In [1]:
# %load /Users/dsuess/Code/Pythonlibs/cluster_template.ipy
import ipyparallel
from os import environ

CLUSTER_ID = environ.get('CLUSTER_ID', None)
_clients = ipyparallel.Client(cluster_id=CLUSTER_ID)
_view = _clients.load_balanced_view()
print("Kernels available: {}".format(len(_clients)))

for c in _clients:
    seed = np.random.randint(10000000)
    c.push({'RGEN': np.random.RandomState(seed=seed)})

Kernels available: 4


In [ ]:
import numpy as np
import matplotlib.pyplot as pl
import itertools as it

import csalgs.lowrank as lr
from tools.helpers import watch_async_view
import h5py

In [ ]:
%%px --local
import numpy as np
from scipy.linalg import svdvals
import csalgs.lowrank as lr


def condition_number(A, threshold=1e-10):
    svals = svdvals(np.asmatrix(A))
    svals_valid = svals[svals > threshold]
    return np.max(svals_valid) / np.min(svals_valid)

In [ ]:
SAMPLES = 10000

with h5py.File('condition_numbers.h5', 'a') as outfile:
    group = outfile.create_group('gaussian_product')
    for rank in [10, 25, 50, 100]:
        _task = _view.map_async(lambda rank: condition_number(lr.random_lowrank_matrix(1000, rank, rgen=RGEN)),
                                it.repeat(rank, SAMPLES))
        watch_async_view(_task)
        cnr_gauss = _task.result()
        pl.hist(cnr_gauss)
        pl.show()
        group[str(rank)] = cnr_gauss

In [ ]:
with h5py.File('condition_numbers.h5', 'a') as outfile:
    try:
        group = outfile.create_group('scaled_cnr')
    except ValueError:
        group = outfile['gaussian_product']
    for rank in [10, 25, 50, 100]:
        _task = _view.map_async(lambda rank: condition_number(lr.random_lowrank_matrix_cnr(1000, rank, condition_scale=[10, 1], rgen=RGEN)),
                                it.repeat(rank, SAMPLES))
        watch_async_view(_task)
        cnr_gauss = _task.result()
        pl.hist(cnr_gauss)
        pl.show()
        group[str(rank)] = cnr_gauss